In [1]:
import sys
import os

import numpy as np
import pandas as pd

sdt_path = ".."
if sdt_path not in sys.path:
    sys.path.insert(1, sdt_path)
    
from sdt import io, brightness

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%load_ext line_profiler

In [6]:
img = np.load(os.path.join("..", "tests", "daostorm_3d", "data_find", "bead_img.npz"))["img"]
local_max = np.load(os.path.join("..", "tests", "daostorm_3d", "data_find", "bead_finder.npz"))["local_max"]
local_max = pd.DataFrame(local_max[:, [1, 3]], columns=["x", "y"])

In [11]:
def make_loc(n=1):
    ret = []
    for i in range(n):
        l = local_max.copy()
        l["frame"] = i
        ret.append(l)
    return pd.concat(ret)

In [27]:
n = 10
loc = make_loc(n)
loc2 = make_loc(n)
frames = [img] * n

In [37]:
%timeit brightness.from_raw_image(loc, frames, 3, 2, engine="numba")

765 µs ± 5.22 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [31]:
%lprun -f brightness.from_raw_image brightness.from_raw_image(loc, frames, 3, 2, engine="numba")

Timer unit: 1e-06 s

Total time: 0.001697 s
File: ../sdt/brightness.py
Function: from_raw_image at line 156

Line #      Hits         Time  Per Hit   % Time  Line Contents
   156                                           def from_raw_image(positions, frames, radius, bg_frame=2, bg_estimator="mean",
   157                                                              pos_columns=pos_columns, engine="numba"):
   158                                               """Determine particle brightness by counting pixel values
   159                                           
   160                                               Around each localization, all brightness values in a  2*`radius` + 1 times
   161                                               square are added up. Additionally, background is locally determined by
   162                                               calculating the mean brightness in a frame of `bg_frame` pixels width
   163                                               a

In [39]:
%timeit brightness.from_raw_image(loc2, frames, 3, 2, engine="python")

36.4 ms ± 531 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
%lprun -f brightness.from_raw_image brightness.from_raw_image(loc2, frames, 3, 2, engine="python")

Timer unit: 1e-06 s

Total time: 0.080619 s
File: ../sdt/brightness.py
Function: from_raw_image at line 184

Line #      Hits         Time  Per Hit   % Time  Line Contents
   184                                           def from_raw_image(positions, frames, radius, bg_frame=2, bg_estimator="mean",
   185                                                              pos_columns=pos_columns, engine="numba"):
   186                                               """Determine particle brightness by counting pixel values
   187                                           
   188                                               Around each localization, all brightness values in a  2*`radius` + 1 times
   189                                               square are added up. Additionally, background is locally determined by
   190                                               calculating the mean brightness in a frame of `bg_frame` pixels width
   191                                               a